# Simulateur d'espace with master plan remove

#### But : Identifier les equipements du masterplan présent sur la dernière iteration de IT_Equipment afin de simuler l'espace disponible lorsque tous les equipements seront retirer

Ce workflow a pour but de simuler l'espace disponible apres avoir retirer les equipements qui figure dans le masterplan sur chacun des racks afin de permettre une projection des équipes dans le déplacement du matériel.

In [ ]:
DROP table IF EXISTS IT_Equipment_x_Master_plan ;
create table IT_Equipment_x_Master_plan
as
select it.ASSET_ID, it.nom, it.SHORT_NAME, it.Modele, it.Fabricant, it.Nom_de_la_salle, it.Nom_de_baie, it.Reperage_ID, it.Hauteur_U, it.Type, mp.SHORT_NAME Master_Plan_name,
  (CASE 
   WHEN mp.short_name is not null and mp.asset_id is null then 'short_name found'
   WHEN mp.short_name is null and mp.asset_id is not null then 'asset_id found'
   WHEN mp.short_name is not null and mp.asset_id is not null then 'found'
   WHEN mp.short_name is null and mp.asset_id is null then 'not found'
   END) Master_plan_status ,
mp.PR_SCOPE, mp.ROOM Master_plan_salle, mp.dalle Master_plan_dalle, mp.RACK_ID Master_plan_rack, mp.Size_U Master_plan_hauteur

 
from `IT_Equipment_v2` it
LEFT JOIN `Master_Plan_Global` mp ON it.ASSET_ID = mp.asset_id or it.short_name = mp.short_name
GROUP by it.ASSET_ID

In [ ]:
drop table if exists IT_Equipment_with_equipment_removed ;
create table IT_Equipment_with_equipment_removed
as
select it.* from `IT_Equipment_v2` it 
INNER JOIN `IT_Equipment_x_Master_plan` mp  ON it.ASSET_ID = mp.asset_id or it.short_name = mp.short_name
where master_plan_status = 'not found'
group by concat(it.Nom_de_la_salle,it.Nom_de_baie, it.Slot_U)

`rack_capacity_map` : cartographie les racks de toutes les salles afin de rajouter la capacité maximale d'un rack défini à 45 unités U  

In [ ]:
drop table if exists rack_capacity_map_mp ;
create table rack_capacity_map_mp
as 
SELECT it.Nom_de_la_salle, it.Nom_de_baie, '-1' as rack_capacity
from `IT_Equipment_with_equipment_removed` it
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie 

In [ ]:
INSERT INTO `rack_capacity_map_mp`
SELECT it2.Nom_de_la_salle, it2.nom, '-1' as rack_capacity from `rack_capacity_map_mp` it 
RIGHT JOIN `IT_Equipment_rack_description` it2 ON concat(it2.Nom_de_la_salle, it2.nom) = concat(it.Nom_de_la_salle, it.nom_de_baie)
where it.nom_de_baie is null

In [ ]:
update `rack_capacity_map_mp` it
inner join `IT_Equipment_rack_description` it2 ON it.Nom_de_la_salle = it2.Nom_de_la_salle and it.nom_de_baie = it2.nom
set it.rack_capacity = it2.Nombre_de_U

`indicator_it_equipment_simulateur_espace` : indicateur de l'espace disponible pour tous lers rack par rapport à la somme des hauteurs de chaque equipment renseigné sur la CMDB

In [ ]:
drop table if exists indicator_it_equipment_simulateur_espace_raw_mp ;

In [ ]:
create table indicator_it_equipment_simulateur_espace_raw_mp
as
SELECT if(it.Nom_de_baie is not null, concat(it.Nom_de_la_salle, it.Nom_de_baie), concat(r.Nom_de_la_salle, r.Nom_de_baie))  key_id,
if(it.Nom_de_la_salle is not null, it.Nom_de_la_salle, r.Nom_de_la_salle) Nom_de_la_salle,
if(it.Nom_de_baie is not null, it.Nom_de_baie, r.Nom_de_baie) Nom_de_baie,
r.rack_capacity, 
if(sum(CEILING(it.Hauteur_U)) is not null, sum(CEILING(it.Hauteur_U)), 0) slot_occupied,
if(sum(CEILING(it.Hauteur_U)) is not null, r.rack_capacity - sum(CEILING(it.Hauteur_U)), r.rack_capacity) free_slot,
if(sum(CEILING(it.Hauteur_U)) is not null, GROUP_CONCAT(concat(it.slot_u, '/', CEILING(it.Hauteur_U))), '') list_occupied_slot
from `IT_Equipment_v2` it
RIGHT JOIN `rack_capacity_map_mp` r ON r.Nom_de_la_salle = it.Nom_de_la_salle and r.Nom_de_baie = it.Nom_de_baie

GROUP BY r.Nom_de_la_salle, r.Nom_de_baie
-- ORDER by r.Nom_de_la_salle desc , sum(CEILING(it.Hauteur_U)) desc

In [ ]:
drop table if exists indicator_it_equipment_simulateur_espace_before_after_mp;
create table indicator_it_equipment_simulateur_espace_before_after_mp
as
select it.key_id, it.Nom_de_la_salle, it.Nom_de_baie, it.free_slot free_slot_before_mp, it.free_series free_series_before_mp, 
if(it2.free_slot is not null, it2.free_slot, it.free_slot) free_slot_after_mp, 
if(it2.free_series is not null, it2.free_series, it.free_series) free_series_after_mp
from `indicator_it_equipment_simulateur_espace` it
LEFT JOIN `indicator_it_equipment_simulateur_espace_mp` it2 ON it.key_id = it2.key_id